#### Este baseline se basara en armar un modelo LSTM por cada producto, con una optimizacion de hiper parametros escueta, para poder comparar con futuros experimientos. En caso de que esta alternativa funcione bien, seria recomendable incorporar parametros de optimizacion extra.

#### Imports

In [59]:
import warnings
warnings.filterwarnings('ignore')
from prophet import Prophet

import pandas as pd
import numpy as np

In [60]:
final_dataset = pd.read_csv('../../Datasets/final_dataset.csv', sep='\t')

In [61]:
final_dataset.head()

,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age
0,20001,201701,0,479,937.72717,934.77222,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0
1,20001,201702,0,432,833.72187,798.01620,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,1
2,20001,201703,0,509,1330.74697,1303.35771,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,1,2
3,20001,201704,0,279,1132.94430,1069.96130,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,3
4,20001,201705,0,701,1550.68936,1502.20132,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,4


In [62]:
columns = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'close_quarter','y']

In [63]:
duplicates = final_dataset.duplicated(subset=['product_id', 'periodo'])
duplicate_rows = final_dataset[duplicates]

if duplicate_rows.empty:
    print("No hay registros duplicados por product_id y periodo.")
else:
    print("Registros duplicados encontrados por product_id y periodo:")
    display(duplicate_rows)


No hay registros duplicados por product_id y periodo.


#### Funcion para crear los modelos

#### Armado de los modelos

In [64]:
final_dataset['periodo'] = final_dataset['periodo'].astype(str)
final_dataset['periodo'] = pd.to_datetime(final_dataset['periodo'], format='%Y%m', errors='coerce')

In [65]:
import os
import joblib
from neuralprophet import NeuralProphet

product_ids = final_dataset['product_id'].unique()
predictions = []
    
for product_id in product_ids:
    product_data = final_dataset[final_dataset['product_id'] == product_id].sort_values(by='periodo')[['periodo', 'y']]
    
    product_data = product_data.rename(columns={'periodo': 'ds', 'y': 'y'})
    # product_data['y'] = np.log1p(product_data['y'])

    try:
        model = NeuralProphet(
            yearly_seasonality=True,
            n_changepoints=5,
            learning_rate=0.01,
            epochs=100,
        )
        model.fit(product_data, freq='M')
    except:
        display(product_data)
    
    future = model.make_future_dataframe(product_data, periods=2)
    forecast = model.predict(future)
    # forecast['yhat'] = np.expm1(forecast['yhat'])

    os.makedirs('NeuralProphet_results', exist_ok=True)
    joblib.dump(model, f'NeuralProphet_results/model_product_{product_id}.pkl')
    print(forecast.head())  # Imprime las primeras filas del DataFrame forecast

    predicted_y = forecast.iloc[-1]['yhat1']
    predictions.append({'product_id': product_id, 'predicted_y': predicted_y})

    print(f'Modelo para el producto {product_id} entrenado y guardado. Predicción a 2 meses: {predicted_y}. Numero de predicciones ${len(predictions)}')

Epoch 100: 100%|██████████| 100/100 [00:00<00:00, 1619.88it/s, loss=0.155, v_num=762, MAE=0.119, RMSE=0.171, Loss=0.160, RegLoss=0.000] 

INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [97.222]% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [50.]% of the data.
WARNING - (NP.df_utils._infer_frequency) - Dataframe has multiple frequencies. It will be resampled according to given freq MS. Ignore                     message if actual frequency is any of the following:  SM, BM, CBM, SMS, BMS, CBMS, BQ, BQS, BA,                         or, BAS.
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [50.]% of the data.
WARNING - (NP.df_utils._infer_frequency) - Dataframe has multiple frequencies. It will be resampled according to given freq MS. Ignore                     message if actual frequency is any of the following:  SM, BM, CBM, SMS, BMS, CBMS, BQ, BQS, BA,                         or, BAS.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 126.75it/s]


INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [97.143]% of the data.


...

INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [90.]% of the data.


INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [50.]% of the data.
WARNING - (NP.df_utils._infer_frequency) - Dataframe has multiple frequencies. It will be resampled according to given freq MS. Ignore                     message if actual frequency is any of the following:  SM, BM, CBM, SMS, BMS, CBMS, BQ, BQS, BA,                         or, BAS.
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [50.]% of the data.
WARNING - (NP.df_utils._infer_frequency) - Dataframe has multiple frequencies. It will be resampled according to given freq MS. Ignore                     message if actual frequency is any of the following:  SM, BM, CBM, SMS, BMS, CBMS, BQ, BQS, BA,                         or, BAS.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 111.10it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column



          ds     y     yhat1     trend  season_yearly
0 2020-01-01  None  0.053906  0.037320       0.016586
1 2020-02-01  None  0.055469  0.034515       0.020954
Modelo para el producto 21276 entrenado y guardado. Predicción a 2 meses: 0.055469200015068054. Numero de predicciones $780


In [67]:
predictions_df = pd.DataFrame(predictions)
predictions_df.to_csv('../../Datasets/predictions.csv', index=False)

print('Todas las predicciones han sido generadas y guardadas en predictions.csv.')

Todas las predicciones han sido generadas y guardadas en predictions.csv.
